In [0]:
%matplotlib inline

import requests

from bs4 import BeautifulSoup as bs

import dataset

from matplotlib import pyplot as plt


import pandas as pd

from datetime import datetime
import time

import bt

ModuleNotFoundError: ignored

# 准备数据

In [0]:
db = dataset.connect("sqlite:///coinmarketcap.db")

NameError: ignored

In [0]:
response = requests.get("https://coinmarketcap.com/")

In [0]:
soup = bs(response.text, 'html.parser')

In [0]:
for tr in soup.find('tbody').find_all("tr"):
    path = tr.find("span").find("a")['href']
    coinname = path.split("/")[-2]
    print(coinname)

#     if len(list(db['price_history'].find(coin = coinname))) >= 360:
#         continue
    

    et = int((datetime(2018,10,30) - datetime(1970,1,1,0)).total_seconds()) * 1000
    st = et - 360 * 24 * 60 * 60 * 1000
    url = f"https://graphs2.coinmarketcap.com{path}{st}/{et}/"
    print(url)

    response = requests.get(url)
    dat = response.json()


    for ts, price in dat['price_usd']:
        db['price_history'].upsert(
            dict(
                ts = ts,
                coin = coinname,
                price = price
            ),
            ['ts','coin']
        )
    time.sleep(2)

In [0]:
del db

# 读取数据

In [0]:
from sqlalchemy import create_engine

In [0]:
import pandas as pd

In [0]:
db_conn = create_engine("sqlite:///coinmarketcap.db")

In [0]:
dat = pd.read_sql("SELECT * FROM price_history", db_conn)

In [0]:
type(dat)

In [0]:
dat.head(10)

In [0]:
df = dat.pivot(index='ts', values='price', columns='coin')

In [0]:
df.head()

In [0]:
df = df.fillna(method="ffill", axis=0)

In [0]:
pd.to_datetime(df.index*1e6)

In [0]:
df.index = pd.to_datetime(df.index*1e6)

In [0]:
df

# 回测

In [0]:
df.head()

In [0]:
s = bt.Strategy('s1', [bt.algos.RunMonthly(),
                       bt.algos.SelectAll(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])

test = bt.Backtest(s,df)

res = bt.run(test)

In [0]:
res.plot()

In [0]:
res.plot()

In [0]:
res.display()

In [0]:
test = bt.Backtest(s, df[['bitcoin','ethereum', 'ripple', 'litecoin', 'bitcoin-cash',]])
res = bt.run(test)

In [0]:
res.plot()

In [0]:
res.display()

In [0]:
res.plot_histogram()

In [0]:
res.plot_security_weights()

In [0]:
sma = df.rolling(50).mean()

In [0]:
signal = df > sma

In [0]:
signal.tail()

In [0]:
sig = signal.ix['2018-10-29 23:59:29']

In [0]:
bool_list = sig
print( len(bool_list), len(sig))
list(sig[bool_list].index)

In [0]:
signal = df < sma

In [0]:

class SelectWhere(bt.Algo):
    def __init__(self, signal):
        self.signal = signal

    def __call__(self, target):
        if target.now in self.signal.index:
            sig = self.signal.ix[target.now]
            selected = list(sig.index[sig])
            target.temp['selected'] = selected # [ 'bitcoin', '0x']
        return True

In [0]:
sub_df = df[['bitcoin','ethereum', 'ripple', 'litecoin', 'bitcoin-cash',]]

sub_sma = sub_df.rolling(50).mean()

sub_signal = sub_df > sub_sma

In [0]:
s = bt.Strategy('s1', [bt.algos.RunMonthly(),
                       SelectWhere(sub_signal),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])

test = bt.Backtest(s, sub_df)

res = bt.run(test)

In [0]:
res.plot()

In [0]:
res.display()

In [0]:
import numpy as np

In [0]:
df = pd.DataFrame({'B': [0, 1, 2, 3, 4]})

In [0]:
df

In [0]:
df.rolling(3).mean()